#Survival Analysis with Python

This notebook demonstrates the basics of survival analysis, a method used for analyzing time to event data, using Python. It has 6 sections. 

1) A brief introduction to survival analysis and the data used in this notebook

2) Non-parametric methods: Kaplan-meier curves, Log-rank test statistic for $\geq2$ groups

3) Semi-parametric methods: Cox proportional hazard model, Schoenfeld residuals, log-log plots

4) Parametric methods: Exponential (accelerated failure time (AFT), proportional hazards (PH)), Weibull (AFT, PH), Gompertz (PH), Log-logistic (proportional odds (PO)), Log-normal (AFT), Generalized Gamma (AFT) 

5) Constructing confidence intervals for survival predictions for the models in section 4

6) Appendix A: Parametric Model Results with Different Optimization Methods

1) A brief introduction to survival analysis and the data used in this notebook

Survival analysis is a collection of methods to analyse time to event data. A key part of survival analysis is the modeling of the contribution of censored observations to the likelihood of model parameters. Censored observations are those for which we have not observed the event (also called right-censoring). The event could be the death of a patient, diagnosis of a disease, failure of a machine part and in this analysis, it is defined as the end of terrorist groups. In the dataset that we will use, not all terrorist groups ended until the end of the observation period. Survival analysis is a suitable method to analyze the factors that influence the time to end of these groups. One very important assumption of survival analysis is that the time to event for censored terror groups does not systematically differ from those whose end we have observed. 

If time to event has the probability density function $f(t)$ and cumulative distribution function $F(t)$, then the probability of surviving at least to time $t$ is: $Pr(T>t)=S(t)=1-F(t)$. Cumulative hazard at time t is defined as $H(t)=-ln(S(t))$ and instantaneous hazard at time $t$ is $h(t)=\frac{dH(t)}{dt}$. The instantateous hazard can also be written as $h(t)=\frac{f(t)}{S(t)}$

The likelihood function for survival analysis is described as:

$$ l(\beta) = \prod_{n=1}^{n} h(t_{i})^{d_{i}} S(t_{i}) $$

where $d_i$ is the censoring variable that equals to 1 if the event is observed for individual $i$ and 0 if the event is not observed (censored) for individual $i$, $h(t_i)$ is the hazard for individual $i$ at time $t$, $H(t_i)$ is the cumulative hazard for individual $i$ at time $t$, and $S(t_i)$ is the survival probability for individual $i$ at time $t$. Note that when $d_i=0$, the contribution of the $i$'th individual to the likelihood function is just its survival probability until time $t$: S(t). If the individual has the event, the contribution to the likelihood function is given by the density function $f(t)=h(t)S(t)$.

The log of likelihood is:

$$ logl(\beta) = \sum_{i=1}^n d_i log(h(t_i)) - H(t_i) $$

where $log$  is the natural logarithm.

The dataset used here is from Jones and Libicki 'How Terrorist Groups End' (Santa Monica, 2008, RAND Corporation). The full report can be found here:

http://www.rand.org/content/dam/rand/pubs/monographs/2008/RAND_MG741-1.pdf

Jones and Libicki use data from RAND - Oklahoma City National Memorial Institute for the Prevention of Terrorism (MIPT) Terrorism Incident database to investigate the factors that contribute to end of terrorist groups. The data includes starting time for 648 terrorist groups that operated between 1968 and 2006. For each of the 648 terrorist groups, there are 5 categorical variables. 

Operating Peak Size: The peak size of the group. The variable can take the followng values; 10,000s (10,000 or more), 1,000s (1,000-9,999), 100s (100-999) and 10s (less than 100).

Econ: The income level of the base country, described according to World Bank criteria in the year in which the group ended or in 2006 if the group has not ended. The variable can take one of four values; 
H = high income (\$10,726 or more, in 2005 gross national income [GNI] per capita, calculated using the World Bank Atlas Method) 
UM = upper middle income (\$3,466–\$10,725)
LM = lower middle income (\$876–\$3,465)
L = low income (\$875 or less)

'Goal' is the primary goal of the terrorist group. It can be one of 'Regime_change','Territorial_change','Policy_change','Empire','Social_revolution','Status_Quo'

'Regime' is the level of freedom in the country against which the terrorist group operates, according to Freedom House classification, based on an assessment of political rights and civil liberties. 'Regime' takes 'Not free', 'Partly free' and 'Free' values.

'Type' is the type of the terrorist group and it takes the following values 'Left_wing','Right_wing','Nationalist', 

'Reason' is the reason for ending, it can take the values 'Policing' 'Splintering' 'Politics' 'Victory' 'Military_force' and 'Not ended'

In addition to the variables described above, we need a 'time' and 'censored' variable to perform the survival analysis. 'Time' variable is the time in years since the start of the terrorist group's activities, either to the time when the terror group ended or to the last observation year (2006) if the terror group did not end until the end of the observed period (1968 - 2006). The censoring variable is labeled 'Ended' and it equals to 1 if the terror group ended between 1968 and 2006 and it equals to 0 if the terror group did not end until 2006.



Before starting the analyses, we need to import the following packages.

In [1]:
#We will use Pandas Dataframe object to hold our data
#and to perform necessary manipulations to prepare the dataset
#for analysis 
import pandas as pd
#%matplotlib inline
%matplotlib inline
#Iport matplotlib.pyplot for plotting results
import matplotlib.pyplot as plt
#Numpy will be used to perform numerical operations on arrays
#(calculate dot products, sums, exponentials, logarithms, find unique values)  
import numpy as np
#We will use scipy to calculate normal 
#distribution values
from scipy.stats import norm
#To set the working directory
import os as osvariable
#To read in csv file
from pandas import read_csv
#Lifelines is a survival analysis package. We will
#use its KaplanMeier curve plotting function,
#logrank_test and Cox proportional hazards fitter
#http://lifelines.readthedocs.org/en/latest/
from lifelines import KaplanMeierFitter
from lifelines.statistics import multivariate_logrank_test   
from lifelines.statistics import logrank_test
from lifelines import CoxPHFitter
#Import the statsmodels. We will use this to 
#fit linear functions to data, which will be 
#helpful to visually assess parametric fits
#http://statsmodels.sourceforge.net/
import statsmodels.api as st
#Genericlikelihood model is what we will use 
#to specify log-likelihood functions for survival
#models: Exponential (accelerated failure time (AFT), proportional hazards (PH)), 
#Weibull (AFT, PH), Gompertz (PH), Log-logistic (proportional odds (PO)), 
#Log-normal (AFT), Generalized Gamma (AFT) 
from statsmodels.base.model import GenericLikelihoodModel
#Import the functions that will be used to calculate the 
#generalized gamma function survival and its confidence
#intervals
#Gamma function
#http://docs.scipy.org/doc/scipy-0.16.0/reference/generated/scipy.stats.gamma.html
from scipy.special import gamma as gammafunction
#Lower regularized incomplete gamma function
#http://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.special.gammainc.html
from scipy.special import gammainc
#Digamma function, which is used when taking the 
#derivative of the gamma function
from scipy.special import psi
#From mpmath library, we will use the meijer G function
#which is part of the derivative of the incomplete gamma function
#http://mpmath.googlecode.com/svn-history/r1229/trunk/doc/build/functions/hypergeometric.html
import mpmath
#from sympy library, we will use the DiracDelta function
#which is part of the derivative of the sign function which in turn
#is part of the generalized gamma function
#http://docs.sympy.org/dev/modules/functions/special.html
from sympy import DiracDelta

Next, set the working directory, load the dataset and prepare the dataset for analysis.

In [2]:
#set working directory:
osvariable.chdir('C:/----/----')
#Read in data
terrordata = read_csv('terrordata1.csv')
#Take a look at the dataset contents, print the first 5 observations
print(terrordata.head())
#Check the categorical variable values
print('Categorical variable values:')
print('Type values:',np.unique(terrordata['Type']))
print('Operating Peak Size values:',np.unique(terrordata['Operating Peak Size']))
print('Regime values:',np.unique(terrordata['Regime']))
print('Goal values:',np.unique(terrordata['Goal']))

   Time  Ended Operating Peak Size Econ. Regime Type Goal    Reason
0     3      0                 10s    LM     NF    R   RC  NotEnded
1     9      1                100s    UM     PF   LW   RC        PO
2    29      0                 10s    LM      F    N   TC         S
3     6      1                 10s     H      F   LW   PC         S
4     7      0                 10s    LM      F    R   PC         S
Categorical variable values:
('Type values:', array(['LW', 'N', 'R', 'RW', 'Reigious'], dtype=object))
('Operating Peak Size values:', array(['1,000s', '1,00s', '10,000s', '100s', '10S', '10c', '10s'], dtype=object))
('Regime values:', array(['BF', 'F', 'NF', 'PF'], dtype=object))
('Goal values:', array(['E', 'PC', 'RC', 'SQ', 'SR', 'TC', 'TCs'], dtype=object))


In [3]:
#One of the entries for 'Type' is entered as 'Reigious'. This
#should be coded as 'R'
terrordata.loc[(terrordata['Type'] == 'Reigious'),['Type']] = 'R'
#Correct the 'Operating Peak Size' variables that are 
#entered incorrectly
terrordata.loc[(terrordata['Operating Peak Size'] == '10S'),['Operating Peak Size']] = '10s'
terrordata.loc[(terrordata['Operating Peak Size'] == '10c'),['Operating Peak Size']] = '10s'
terrordata.loc[(terrordata['Operating Peak Size'] == '1,00s'),['Operating Peak Size']] = '1,000s'
#One of the entries for 'Regime' is entered incorrectly as 'BF'
terrordata.loc[(terrordata['Regime'] == 'BF'),['Regime']] = 'NF'
#One of the entries for 'Goal' is entered incorrectly as 'TCs'
terrordata.loc[(terrordata['Goal'] == 'TCs'),['Goal']] = 'TC'
#Check the categorical variable values again
print(np.unique(terrordata['Type']))
print(np.unique(terrordata['Operating Peak Size']))
print(np.unique(terrordata['Regime']))
print(np.unique(terrordata['Goal']))

['LW' 'N' 'R' 'RW']
['1,000s' '10,000s' '100s' '10s']
['F' 'NF' 'PF']
['E' 'PC' 'RC' 'SQ' 'SR' 'TC']


In [4]:
#Take a look at the unique values for categorical variables
#Check the categorical variable values
print('Categorical variable values:')
print('Type values:',np.unique(terrordata['Type']))
print('Operating Peak Size values:',np.unique(terrordata['Operating Peak Size']))
print('Regime values:',np.unique(terrordata['Regime']))
print('Goal values:',np.unique(terrordata['Goal']))

Categorical variable values:
('Type values:', array(['LW', 'N', 'R', 'RW'], dtype=object))
('Operating Peak Size values:', array(['1,000s', '10,000s', '100s', '10s'], dtype=object))
('Regime values:', array(['F', 'NF', 'PF'], dtype=object))
('Goal values:', array(['E', 'PC', 'RC', 'SQ', 'SR', 'TC'], dtype=object))


In [5]:
#Replace abbreviations with words to make reading tables easier
terrordata.loc[terrordata['Type'] == 'R',['Type']] = 'Religious'
terrordata.loc[terrordata['Type'] == 'LW',['Type']] = 'Left_wing'
terrordata.loc[terrordata['Type'] == 'N',['Type']] = 'Nationalist'
terrordata.loc[terrordata['Type'] == 'RW',['Type']] = 'Right_wing'

terrordata.loc[terrordata['Regime'] == 'F',['Regime']] = 'Free'
terrordata.loc[terrordata['Regime'] == 'PF',['Regime']] = 'Partly_free'
terrordata.loc[terrordata['Regime'] == 'NF',['Regime']] = 'Not_free'

terrordata.loc[terrordata['Goal'] == 'RC',['Goal']] = 'Regime_change'
terrordata.loc[terrordata['Goal'] == 'TC',['Goal']] = 'Territorial_change'
terrordata.loc[terrordata['Goal'] == 'PC',['Goal']] = 'Policy_change'
terrordata.loc[terrordata['Goal'] == 'E',['Goal']] = 'Empire'
terrordata.loc[terrordata['Goal'] == 'SR',['Goal']] = 'Social_revolution'
terrordata.loc[terrordata['Goal'] == 'SQ',['Goal']] = 'Status_Quo'

terrordata.loc[terrordata['Econ.'] == 'L',['Econ.']] = 'Low_income'
terrordata.loc[terrordata['Econ.'] == 'LM',['Econ.']] = 'Lower_middle_income'
terrordata.loc[terrordata['Econ.'] == 'UM',['Econ.']] = 'Upper_middle_income'
terrordata.loc[terrordata['Econ.'] == 'H',['Econ.']] = 'High_income'

terrordata.loc[terrordata['Reason'] == 'PO',['Reason']] = 'Policing'
terrordata.loc[terrordata['Reason'] == 'S',['Reason']] = 'Splintering'
terrordata.loc[terrordata['Reason'] == 'PT',['Reason']] = 'Politics'
terrordata.loc[terrordata['Reason'] == 'V',['Reason']] = 'Victory'
terrordata.loc[terrordata['Reason'] == 'MF',['Reason']] = 'Military_force'

#Now print the variable names
print(terrordata.columns)

Index([u'Time', u'Ended', u'Operating Peak Size', u'Econ.', u'Regime', u'Type', u'Goal', u'Reason'], dtype='object')


In [6]:
#Create dummy variables for categorical variables
#Store dummy variables for each variable
sizevars = pd.get_dummies(terrordata['Operating Peak Size'])
econvars = pd.get_dummies(terrordata['Econ.'])
regimevars = pd.get_dummies(terrordata['Regime'])
typevars = pd.get_dummies(terrordata['Type'])
goalvars = pd.get_dummies(terrordata['Goal'])
reasonvars = pd.get_dummies(terrordata['Reason'])

#Add all dummy variables to the original dataset
for var in sizevars:
    terrordata[var] = sizevars[var]
for var in econvars:
    terrordata[var] = econvars[var]
for var in regimevars:
    terrordata[var] = regimevars[var]
for var in typevars:
    terrordata[var] = typevars[var]
for var in goalvars:
    terrordata[var] = goalvars[var]
for var in reasonvars:
    terrordata[var] = reasonvars[var]
    
#The dataset now includes all variables and their dummies
print(terrordata.columns)  

Index([u'Time', u'Ended', u'Operating Peak Size', u'Econ.', u'Regime', u'Type', u'Goal', u'Reason', u'1,000s', u'10,000s', u'100s', u'10s', u'High_income', u'Low_income', u'Lower_middle_income', u'Upper_middle_income', u'Free', u'Not_free', u'Partly_free', u'Left_wing', u'Nationalist', u'Religious', u'Right_wing', u'Empire', u'Policy_change', u'Regime_change', u'Social_revolution', u'Status_Quo', u'Territorial_change', u'Military_force', u'NotEnded', u'Policing', u'Politics', u'Splintering', u'Victory'], dtype='object')


In [7]:
#Create the dataframe that we will use for analyses.
#Because we have categorical variables, we will leave 
#one dummy variable from each categorical variable out 
#as the reference case. Note that we are leaving
#variables for 'reason' out, since one of the categories
#of this variable ('not ended') matches the '0' value of the 
#'Event' variable 

#Reference categories that are left out are 
#'Regime_change', '10,000s', 'High_income'
#'Not_free', 'Left_wing'.
survivaldata = terrordata[['Territorial_change','Policy_change','Empire','Social_revolution','Status_Quo','1,000s','100s','10s','Low_income','Lower_middle_income','Upper_middle_income','Partly_free','Free','Nationalist','Religious','Right_wing']]    

#Add a constant term to the data
survivaldata = st.add_constant(survivaldata, prepend=False)

#Create the event variable. 'Ended' equals 1 if the terrorist group has 
#ended within the observation period and to 0 if it did not
eventvar = terrordata['Ended']

#Create the time variable. Time is in years and it is assumed that the minimum
#value it takes is 1
timevar = terrordata['Time']

The dataset is ready for analysis. We start with non-parametric, semi-parametric and parametric methods.